# **Data Cleaning and Formatting for Analysis**

This notebook contains the steps to clean and format the data, making it ready for further analysis.

## **Load and display initial data**

### **Importing necessary libraries**

In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import json

### **Load the dataset**

In [84]:
# Read the CSV file into a DataFrame
match_df = pd.read_csv('./data/data02.csv')

# Display the first few rows to understand the structure of the dataset
match_df.head()

,puuid,augments_1,augments_2,augments_3,traits_1_name,traits_2_name,traits_3_name,traits_4_name,traits_5_name,traits_6_name,...,traits_15_num_units,traits_15_style,traits_15_tier_current,traits_15_tier_total,traits_16_num_units,traits_16_style,traits_16_tier_current,traits_16_tier_total,level,placement
0,V4CYQ37ANOdyBmU8dxoT4HsPWHkOSY2oaijwrr6Ijj7Aoe...,TFT11_Augment_StoryweaverCrest,TFT9_Augment_StationarySupport3,TFT6_Augment_OneTwoFive,TFT11_Altruist,TFT11_Arcanist,TFT11_Behemoth,TFT11_Bruiser,TFT11_Dryad,TFT11_Exalted,...,0,0,0,0,0,0,0,0,8,7
1,4RHYO23SWrm7h35kCmesJOqErQ8Gh0uoUuUYKvqrJIxoXe...,TFT11_Augment_Trickshot,TFT11_Augment_TrashToTreasure,TFT9_Augment_Commander_RollingForDays,TFT11_Arcanist,TFT11_Bruiser,TFT11_Dragonlord,TFT11_Exalted,TFT11_Fated,TFT11_Fortune,...,0,0,0,0,0,0,0,0,9,4
2,orvNdZ6YNzS5oHAvPlKFGzp0j5-z-xQg4pT0uFA4bb7eWc...,TFT9_Augment_Idealism,TFT6_Augment_GachaAddict,TFT9_Augment_RedBuff,TFT11_Dragonlord,TFT11_Duelist,TFT11_Fortune,TFT11_Great,TFT11_Heavenly,TFT11_InkShadow,...,0,0,0,0,0,0,0,0,8,6
3,RgqCZQcvEAIq0VhCVwyrek4aDuait82v8P9c6mHDjvQ5pE...,TFT9_Augment_YouHaveMyBow,TFT11_Augment_Accomplice,TFT9_Augment_Harmacist1,TFT11_Arcanist,TFT11_Behemoth,TFT11_Exalted,TFT11_Fated,TFT11_Ghostly,TFT11_Mythic,...,0,0,0,0,0,0,0,0,7,8
4,uQg8ITKGu-ywgNI3pzWjqTLFFGyRHE0aHwtKAsixZTkACn...,TFT9_Augment_Sleightofhand,TFT9_Augment_StationarySupport3,TFT9_Augment_CyberneticBulk1,TFT11_Arcanist,TFT11_Artist,TFT11_Behemoth,TFT11_Bruiser,TFT11_Dryad,TFT11_Duelist,...,2,1,1,4,1,5,1,1,10,1


In [85]:
# Drop the column 'id'
match_df = match_df.drop(columns=['puuid'])

## **Formatting Names**

In this section, we format the character names to remove underscores and capitalize them properly.

In [86]:
# Function to format character names of augments
def format_augment_name(augment):
    # Split the string by '_'
    parts = augment.split('_')
    # Get the last element
    formatted_name = parts[-1]
    # Use regular expression to split each word
    formatted_name = ' '.join(re.findall('[A-Z][^A-Z]*', formatted_name))

    return formatted_name

# Format data in the 3 columns augments_1, augments_2, augments_3
for col in ['augments_1', 'augments_2', 'augments_3']:
    match_df[col] = match_df[col].apply(format_augment_name)

In [87]:
# Function to format character names of traits
def format_trait_name(trait):
    # Split the string by '_'
    parts = trait.split('_')
    # Get the last element
    formatted_name = parts[-1]
    return formatted_name

# Format data in the 16 columns traits
for i in range(1, 17):
    col = f'traits_{i}_name'
    match_df[col] = match_df[col].apply(format_trait_name)

In [88]:
# Function to format character names of champions
def format_character_name(units):
    # Split the string by '_'
    parts = units.split('_')
    # Get the last element
    formatted_name = parts[-1]
    return formatted_name

# Format data in the 16 columns of champion names
for i in range(1, 11):
    col = f'units_{i}_character_id'
    match_df[col] = match_df[col].apply(format_character_name)

# Rename columns
for i in range(1, 11):
    old_col_name = f'units_{i}_character_id'
    new_col_name = f'units_{i}_name'
    match_df.rename(columns={old_col_name: new_col_name}, inplace=True)

In [89]:
# Function to format character names of items
def format_item_name(item):
    # Split the string by '_'
    parts = item.split('_')
    # Get the last element
    formatted_name = parts[-1]
    # Use regular expression to split each word
    formatted_name = ' '.join(re.findall('[A-Z][^A-Z]*', formatted_name))
    return formatted_name

# Format data in the columns units_{1-10}_item_{1-3}
for unit in range(1, 11):
    for item in range(1, 4):
        col_name = f'units_{unit}_item_{item}'
        match_df[col_name] = match_df[col_name].apply(format_item_name)


## **Data Segmentation**

Here, we split the dataset into smaller segments for focused analysis, including augments, traits, and units.

In [90]:
# Augments
augments = match_df[['augments_1', 'augments_2', 'augments_3', 'placement']]

# Traits
traits_df = match_df[[f'traits_{i}_name'for i in range(1, 17)] +
                     [f'traits_{i}_num_units' for i in range(1, 17)] +
                     [f'traits_{i}_style' for i in range(1, 17)] +
                     [f'traits_{i}_tier_current' for i in range(1, 17)] +
                     [f'traits_{i}_tier_total' for i in range(1, 17)]]

# Units
units_df = match_df[[f'units_{i}_name' for i in range(1, 11)] +
                    [f'units_{i}_item_{j}' for i in range(1, 11) for j in range(1, 4)]]

## **Numeric Encoding**

### **Augments Name Encoding**

In [91]:
# Read data in standard name format of augments
id_name = pd.read_csv('data/Id_augment.csv')
name_type = pd.read_csv('data/Type.csv')

In [92]:
# Create a mapping from ID to name
mapping_name = id_name.to_dict('records')
mapping_name = {row['id']: row['name'] for row in mapping_name}

# Create a mapping from augment name to type
mapping_type = name_type.to_dict('records')
mapping_type = {row['Augment'].strip(): row['Type'] for row in mapping_type}

In [93]:
# Read available JSON data to retrieve data for converting IDs of augments to right names
with open('data/Name.json', 'r') as f:
    temp = json.load(f)

names = dict()
for i in temp['data']:
    names[i] = temp['data'][i]['name']


In [94]:
def replace_suffix(x:str):
    x = x.replace('III','').replace('II','').replace('++','').replace('+','').strip()
    if x.endswith('I'):
        return x[:-1].strip()
    return x
def update_value(val):
    if val in names:
        return names[val]
    elif val in mapping_name:
        return mapping_name[val]
    else:
        return val

In [95]:
augments = augments.map(update_value)
augments = augments.map(lambda x: str(x).replace('++','').replace('+','').strip())

In [96]:
all_augments = mapping_type.keys()
all_augments = list(set(map(replace_suffix, all_augments)))
all_augments.sort()

In [97]:
augments.replace('Healing Orbs I I', 'Healing Orbs II', inplace=True)

In [98]:
# Creat new dataframe for all augments
augments_df = pd.DataFrame(0, index=augments.index, columns=all_augments)

for i in range(len(augments)):
    for j in range(3):
        ag = augments[f'augments_{j + 1}'][i]
        if ag.endswith('I'):
            count = ag.count('I')
            ag = replace_suffix(ag)
            augments_df.loc[i, ag] += count
        elif ag in mapping_type:
            augments_df.loc[i, ag] += mapping_type[ag]

### **Traits Name Encoding**

In [99]:
# Process for traits_{i}_name for i from 1 to 16
trait_name_columns = [f'traits_{i}_name' for i in range(1, 17)]
traits_df = traits_df[trait_name_columns]

# Get all unique traits names
unique_traits_names = pd.unique(traits_df.values.ravel('K'))

# Create a new dataframe with columns for each unique trait name
traits_encoding_df = pd.DataFrame(0, index=traits_df.index, columns=unique_traits_names)

# Populate the new dataframe by setting 1 if the trait name appears in any of the trait name columns for a row in traits_df

check = True
for idx, row in traits_df.iterrows():
    for col in trait_name_columns:
        tier = col.replace('name', '') + 'tier_current'
        if pd.notna(row[col]):
            traits_encoding_df.at[idx, row[col]] = match_df[tier][idx]

### **Units Name Encoding**

In [100]:
# Process for units_k_name where k ranges from 1 to 10
unit_name_columns = [f'units_{i}_name' for i in range(1, 11)]
unique_unit_names = pd.unique(units_df[unit_name_columns].values.ravel('K'))

# Create a new dataframe with columns for each unique unit name
units_encoding_df = pd.DataFrame(0, index=units_df.index, columns=unique_unit_names)

# Populate the new dataframe by setting its tier if the unit name appears in any of the unit name columns for a row in units_df
check = True
for idx, row in units_df.iterrows():
    for col in unit_name_columns:
        tier = col.replace('name', '') + 'tier'
        if pd.notna(row[col]):
            units_encoding_df.at[idx, row[col]] = match_df[tier][idx]

### **Item Name Encoding**

In [101]:
# Define the correct columns for unit items based on the provided pattern
unit_item_columns = [f'units_{i}_item_{j}' for i in range(1, 11) for j in range(1, 4)]

# Filter out columns that don't exist in the dataframe
unit_item_columns = [col for col in unit_item_columns if col in units_df.columns]

# Extract all unique items from these columns
unique_unit_items = pd.unique(units_df[unit_item_columns].values.ravel('K'))
unique_unit_items = [item for item in unique_unit_items if not pd.isnull(item)]

# Create a new dataframe with columns for each unique unit item
units_items_df = pd.DataFrame(0, index=units_df.index, columns=unique_unit_items)

# Populate the new dataframe
for idx, row in units_df.iterrows():
    for col in unit_item_columns:
        if pd.notna(row[col]):
            units_items_df.at[idx, row[col]] = 1

In [102]:
remaining = match_df[match_df.columns.difference(augments.columns).difference(traits_df.columns).difference(units_df.columns)]
drop_columns = [x for x in list(remaining.columns) if (x.find('tier') != -1 or x.find('style') != -1 or x.find('_num_units') != -1)]
df = remaining.copy()
df.drop(columns=drop_columns, inplace=True)

This cell calculates the remaining columns that have not been included in the previously defined augments, traits, and units DataFrames. It then drops any columns related to 'tier', 'style', or '_num_units' that are deemed irrelevant for the final analysis.

In [103]:
remaining = df

## **Final DataFrame**

In [104]:
final_data = pd.concat([remaining, augments_df, traits_encoding_df, units_encoding_df, units_items_df, match_df['placement']], axis=1)

In [105]:
final_data.drop(columns=['0', 'Empty Bag'], inplace=True) # Remove 2 meaningless columns of meaningless

In [108]:
final_data.drop(columns=['Artist', 'Great', 'Lovers', 'SpiritWalker'], inplace=True) # Remove columns of traits which has only one champion

In [109]:
final_data

,level,A Cut Above,Accomplice,Altruist Crest,Altruist Crown,Arcanist,Arcanist Crest,Arcanist Crown,Ascension,AtWhatCost,...,Bramble Vest Radiant,Steraks Gage Radiant,Spectral Gauntlet Radiant,Jeweled Gauntlet Radiant,Quicksilver Radiant,Guardian Angel Radiant,A P,Support,A S,placement
0,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,7
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
3,7,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9587,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
9588,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9589,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
9590,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [110]:
final_data.to_csv('data/final.csv', index=False, header=True)